In [1]:
import torch
import torch.nn as nn
import numpy as np
import joblib
from torch_geometric.data import Data

In [2]:
patient_and_stage = joblib.load("esca_patients_and_stage.pkl")

In [3]:
len(patient_and_stage)

135

In [26]:
patient_and_stage['TCGA-LN-A4MR']

'Stage IIA'

In [5]:
tmp_stage = []
for x in patient_and_stage:
    tmp_stage.append(patient_and_stage[x])
print(set(tmp_stage))

{'Stage IIIB', 'Stage IIIC', 'Stage IIA', 'Stage IV', 'Stage IB', 'Stage IVA', 'Stage III', 'Stage I', 'Stage IIB', 'Stage IA', 'Stage II', 'Stage IIIA'}


In [6]:
patient_and_label = {}
for x in patient_and_stage:
    if patient_and_stage[x] in ['Stage III', 'Stage IIIB', 'Stage IIIC', 'Stage IV', 'Stage IVA', 'Stage IIIA']:
        patient_and_label[x] = 1
    elif patient_and_stage[x] in [ 'Stage IIB', 'Stage IA', 'Stage IB', 'Stage II', 'Stage I', 'Stage IIA']:
        patient_and_label[x] = 0

In [27]:
patient_and_label['TCGA-LN-A4MR']

0

In [8]:
patients = []
for x in patient_and_label:
    patients.append(x)

In [9]:
no_sort_t_all_feature = joblib.load("esca_5x10x_kimianet_patch.pkl")

In [28]:
#pretrain feature
no_sort_t_all_feature['TCGA-LN-A4MR']['30-26.png']

array([ 0.11480963,  0.15030876,  0.15438452, ..., -0.3086485 ,
        2.0952566 ,  1.0193117 ], dtype=float32)

In [11]:
t_all_feature = {}
for x in patients:
    feature = {}
    for z in no_sort_t_all_feature[x]:
        if z == '-1':
            feature[z] = no_sort_t_all_feature[x][z]
            
    for z in no_sort_t_all_feature[x]:            
        if len(z.split('-')) == 6:
            feature[z] = no_sort_t_all_feature[x][z]
        
    for z in no_sort_t_all_feature[x]:            
        if len(z.split('-')) == 2 and z != '-1':
            feature[z] = no_sort_t_all_feature[x][z]
        
    t_all_feature[x] = feature


In [34]:
"""
Thumbnail:"-1"
5x:"x1-x2-y1-y2-x-y",x/y are the coordinates of the 5x node, and x1/x2/y1/y2 are the coordinates of the corresponding 10x node
10x:"x-y"
"""
t_all_feature['TCGA-LN-A4MR']['-1']

array([ 0.03040453,  0.26700664,  0.07548372, ..., -0.4094244 ,
        0.886226  ,  1.8047752 ], dtype=float32)

In [32]:
t_all_feature['TCGA-LN-A4MR']['34-35-28-29-17-14.png']

array([0.06940576, 0.23001698, 0.16695607, ..., 0.5770061 , 2.1174974 ,
       0.1034441 ], dtype=float32)

In [33]:
t_all_feature['TCGA-LN-A4MR']['24-23.png']

array([ 0.10520199,  0.12185413,  0.2064581 , ..., -1.242542  ,
        1.4516828 ,  0.48982158], dtype=float32)

In [13]:
all_feature = {}
for x in patients:
    feature = []
    for z in t_all_feature[x]:
        if z == '-1':
            feature.append(t_all_feature[x][z])
    for z in t_all_feature[x]:            
        if len(z.split('-')) == 6:
            feature.append(t_all_feature[x][z])
    for z in t_all_feature[x]:            
        if len(z.split('-')) == 2 and z != '-1':
            feature.append(t_all_feature[x][z])
        
    all_feature[x] = feature


In [35]:
all_feature['TCGA-LN-A4MR'][:5]

[array([ 0.03040453,  0.26700664,  0.07548372, ..., -0.4094244 ,
         0.886226  ,  1.8047752 ], dtype=float32),
 array([0.06940576, 0.23001698, 0.16695607, ..., 0.5770061 , 2.1174974 ,
        0.1034441 ], dtype=float32),
 array([ 0.08397469,  0.17341468, -0.10069387, ...,  0.6422272 ,
         2.5842233 , -1.0786229 ], dtype=float32),
 array([ 0.06828591,  0.23206148,  0.12968677, ...,  1.4263378 ,
         1.6484534 , -0.05423415], dtype=float32),
 array([ 0.09137766,  0.14499718,  0.04132989, ...,  0.58380806,
         0.5065441 , -1.2403352 ], dtype=float32)]

In [15]:

def get_edge_index_2(id):  
    
    start = []
    end = []
    
    patch_id = {}
    i=0
    for x in t_all_feature[id]:
        patch_id[x.split('.')[0]] = i
        i+=1
#     print(patch_id)

    # Edges between nodes of different resolutions
    for x in patch_id:
        if len(x.split('-'))==6:
            start.append(patch_id[x])
            end.append(patch_id['-1'])
            end.append(patch_id[x])
            start.append(patch_id['-1']) 
            
            if x.split('-')[0]+'-'+x.split('-')[2] in patch_id:
                start.append(patch_id[x])
                end.append(patch_id[x.split('-')[0]+'-'+x.split('-')[2]])
                end.append(patch_id[x])
                start.append(patch_id[x.split('-')[0]+'-'+x.split('-')[2]])                
            if x.split('-')[1]+'-'+x.split('-')[2] in patch_id:                
                start.append(patch_id[x])
                end.append(patch_id[x.split('-')[1]+'-'+x.split('-')[2]]) 
                end.append(patch_id[x])
                start.append(patch_id[x.split('-')[1]+'-'+x.split('-')[2]])             
            if x.split('-')[0]+'-'+x.split('-')[3] in patch_id:            
                start.append(patch_id[x])
                end.append(patch_id[x.split('-')[0]+'-'+x.split('-')[3]])
                end.append(patch_id[x])
                start.append(patch_id[x.split('-')[0]+'-'+x.split('-')[3]])            
            if x.split('-')[1]+'-'+x.split('-')[3] in patch_id:            
                start.append(patch_id[x])
                end.append(patch_id[x.split('-')[1]+'-'+x.split('-')[3]])
                end.append(patch_id[x])
                start.append(patch_id[x.split('-')[1]+'-'+x.split('-')[3]]) 


        
    patch_id_5x = {}
    for x in patch_id:
        if len(x.split('-'))==6:
            patch_id_5x[x.split('-')[-2]+'-'+x.split('-')[-1]] = patch_id[x]
#     print(patch_id_5x)

    #5x resolution edge between nodes
    for x in patch_id:
        if len(x.split('-'))==6:
            i = int(x.split('-')[-2])
            j = int(x.split('-')[-1])
#             print(i,j)
            if str(i)+'-'+str(j+1) in patch_id_5x:
                start.append(patch_id_5x[str(i)+'-'+str(j)])
                end.append(patch_id_5x[str(i)+'-'+str(j+1)])
            if str(i)+'-'+str(j-1) in patch_id_5x:
                start.append(patch_id_5x[str(i)+'-'+str(j)])
                end.append(patch_id_5x[str(i)+'-'+str(j-1)])
            if str(i+1)+'-'+str(j) in patch_id_5x:
                start.append(patch_id_5x[str(i)+'-'+str(j)])
                end.append(patch_id_5x[str(i+1)+'-'+str(j)])
            if str(i-1)+'-'+str(j) in patch_id_5x:
                start.append(patch_id_5x[str(i)+'-'+str(j)])
                end.append(patch_id_5x[str(i-1)+'-'+str(j)])
            if str(i+1)+'-'+str(j+1) in patch_id_5x:
                start.append(patch_id_5x[str(i)+'-'+str(j)])
                end.append(patch_id_5x[str(i+1)+'-'+str(j+1)])
            if str(i-1)+'-'+str(j+1) in patch_id_5x:
                start.append(patch_id_5x[str(i)+'-'+str(j)])
                end.append(patch_id_5x[str(i-1)+'-'+str(j+1)])
            if str(i+1)+'-'+str(j-1) in patch_id_5x:
                start.append(patch_id_5x[str(i)+'-'+str(j)])
                end.append(patch_id_5x[str(i+1)+'-'+str(j-1)])
            if str(i-1)+'-'+str(j-1) in patch_id_5x:
                start.append(patch_id_5x[str(i)+'-'+str(j)])
                end.append(patch_id_5x[str(i-1)+'-'+str(j-1)])
                
    #10x resolution edge between nodes
    for x in patch_id:
        if len(x.split('-'))==2 and x!='-1':
#             print(x)
            i = int(x.split('-')[0])
            j = int(x.split('-')[1])
#             print(i,j)
            if str(i)+'-'+str(j+1) in patch_id:
                start.append(patch_id[str(i)+'-'+str(j)])
                end.append(patch_id[str(i)+'-'+str(j+1)])
            if str(i)+'-'+str(j-1) in patch_id:
                start.append(patch_id[str(i)+'-'+str(j)])
                end.append(patch_id[str(i)+'-'+str(j-1)])
            if str(i+1)+'-'+str(j) in patch_id:
                start.append(patch_id[str(i)+'-'+str(j)])
                end.append(patch_id[str(i+1)+'-'+str(j)])
            if str(i-1)+'-'+str(j) in patch_id:
                start.append(patch_id[str(i)+'-'+str(j)])
                end.append(patch_id[str(i-1)+'-'+str(j)])
            if str(i+1)+'-'+str(j+1) in patch_id:
                start.append(patch_id[str(i)+'-'+str(j)])
                end.append(patch_id[str(i+1)+'-'+str(j+1)])
            if str(i-1)+'-'+str(j+1) in patch_id:
                start.append(patch_id[str(i)+'-'+str(j)])
                end.append(patch_id[str(i-1)+'-'+str(j+1)])
            if str(i+1)+'-'+str(j-1) in patch_id:
                start.append(patch_id[str(i)+'-'+str(j)])
                end.append(patch_id[str(i+1)+'-'+str(j-1)])
            if str(i-1)+'-'+str(j-1) in patch_id:
                start.append(patch_id[str(i)+'-'+str(j)])
                end.append(patch_id[str(i-1)+'-'+str(j-1)])               
                
                  
    
#     print(start)
#     print(end)
    return [start,end]  

In [16]:
get_edge_index_2('TCGA-LN-A4MR')[0][:20],get_edge_index_2('TCGA-LN-A4MR')[1][:20]

([1, 0, 1, 300, 1, 261, 2, 0, 2, 364, 2, 292, 2, 557, 2, 201, 3, 0, 3, 137],
 [0, 1, 300, 1, 261, 1, 0, 2, 364, 2, 292, 2, 557, 2, 201, 2, 0, 3, 137, 3])

In [17]:
all_node_type = {}
for x in patients:
    t = []   
    for z in t_all_feature[x]:
        if z == '-1':
            t.append(0)
    for z in t_all_feature[x]:            
        if len(z.split('-')) == 6:
            t.append(1)
    for z in t_all_feature[x]:            
        if len(z.split('-')) == 2 and z != '-1':
            t.append(2)        
      
    all_node_type[x] = t

In [36]:
"""
Thumbnail: 0
5x: 1
10x: 2
"""
np.array(all_node_type['TCGA-LN-A4MR'])

array([0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,

In [19]:
tree = {}
for p_id in patients:
    
    t_tree = np.array([-2]*len(all_node_type[p_id]))
    t_tree[np.where(np.array(all_node_type[p_id])==0)] = -1
    t_tree[np.where(np.array(all_node_type[p_id])==1)] = np.where(np.array(all_node_type[p_id])==0)[0][0]
#     print(t_tree)
    
    patch_id = {}
    i=0
    for x in t_all_feature[p_id]:
        patch_id[x.split('.')[0]] = i
        i+=1
#     print(patch_id)

    for x in patch_id:
        if len(x.split('-'))==6:
            if x.split('-')[0]+'-'+x.split('-')[2] in patch_id:
                t_tree[patch_id[x.split('-')[0]+'-'+x.split('-')[2]]] =  patch_id[x]           
            if x.split('-')[1]+'-'+x.split('-')[2] in patch_id:                
                t_tree[patch_id[x.split('-')[1]+'-'+x.split('-')[2]]] =  patch_id[x]              
            if x.split('-')[0]+'-'+x.split('-')[3] in patch_id:            
                t_tree[patch_id[x.split('-')[0]+'-'+x.split('-')[3]]] =  patch_id[x]            
            if x.split('-')[1]+'-'+x.split('-')[3] in patch_id:            
                t_tree[patch_id[x.split('-')[1]+'-'+x.split('-')[3]]] =  patch_id[x]   
    

    tree[p_id] = t_tree
#     break

In [20]:
tree['TCGA-LN-A4MR']

array([ -1,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,  42,  93,  93,  32,  34,   3,  80,  33, 119,   4,  85,
       119,  26,  42,  83, 107, 120,  60, 123,  17,   4, 122,  28, 125,
        76,  18,  52, 107, 107,  57,  56, 101,  36, 122,  87,  51,  81,
        64,  17,  56,  98,  82,  86,  82,  89, 120,  44,  82,  1

In [21]:
x_y = {}
for x in patients:
    dd = []
    for z in t_all_feature[x]:
        if z == '-1':
            dd.append((0,0))
            
    for z in t_all_feature[x]:            
        if len(z.split('-')) == 6:
            dd.append((float(z.split('-')[-2]),float(z.split('-')[-1].split('.')[0])))
        
    for z in t_all_feature[x]:            
        if len(z.split('-')) == 2 and z != '-1':
            dd.append((float(z.split('-')[0]),float(z.split('-')[1].split('.')[0])))
        
    x_y[x] = dd


In [22]:
max_x_y_5x = {}
max_x_y_10x = {}
for p_id in patients:
    x=[]
    y=[]
    for z in np.array(x_y[p_id])[np.where(np.array(all_node_type[p_id])==1)]:
        x.append(z[0])
        y.append(z[1])
    max_x_y_5x[p_id] = [np.max(np.array(x)),np.max(np.array(y))]
    
    x=[]
    y=[]
    for z in np.array(x_y[p_id])[np.where(np.array(all_node_type[p_id])==2)]:
        x.append(z[0])
        y.append(z[1])
    max_x_y_10x[p_id] = [np.max(np.array(x)),np.max(np.array(y))]    
    

In [23]:
x_y = {}
for x in patients:
    dd = []
    for z in t_all_feature[x]:
        if z == '-1':
            dd.append((0,0))
            
    for z in t_all_feature[x]:            
        if len(z.split('-')) == 6:
            dd.append((float(z.split('-')[-2])/max_x_y_5x[x][0],float(z.split('-')[-1].split('.')[0])/max_x_y_5x[x][1]))
        
    for z in t_all_feature[x]:            
        if len(z.split('-')) == 2 and z != '-1':
            dd.append((float(z.split('-')[0])/max_x_y_10x[x][0],float(z.split('-')[1].split('.')[0])/max_x_y_10x[x][1]))
        
    x_y[x] = dd


In [37]:
#Normalize
x_y['TCGA-LN-A4MR'][:5]

[(0, 0),
 (0.9444444444444444, 0.9333333333333333),
 (0.05555555555555555, 0.2),
 (0.7777777777777778, 0.8666666666666667),
 (0.4444444444444444, 0.5333333333333333)]

In [ ]:

all_data = {}
for id in patients:
    print(id)
    node_attr=torch.tensor(all_feature[id],dtype=torch.float)
    edge_index_tree_8nb = torch.tensor(get_edge_index_2(id),dtype=torch.long) 
    batch = torch.tensor([0 for i in range(len(node_attr))])
    node_type = torch.tensor(all_node_type[id])
    node_tree = torch.tensor(tree[id])
    x_y_index = torch.tensor(x_y[id])
    data_id = id 
    data=Data(x=node_attr,edge_index_tree_8nb=edge_index_tree_8nb,data_id=data_id,batch =batch,node_type=node_type,node_tree=node_tree,x_y_index=x_y_index) 
    all_data[id] = data
#     print(data)
    

In [38]:
all_data['TCGA-LN-A4MR']

Data(batch=[564], data_id="TCGA-LN-A4MR", edge_index_tree_8nb=[2, 5054], node_tree=[564], node_type=[564], x=[564, 1024], x_y_index=[564, 2])

In [ ]:
# joblib.dump(all_data,'esca_5x10x_tree_8nb_gnn_data.pkl')